In [1]:
from PIL import Image
import openslide
import glob
import cv2 as cv
import csv
import numpy as np
from stopwatch import Stopwatch
from sklearn.svm import SVC
import lymph_BoG as bow
import imp
from matplotlib import pyplot as plt
import numpy as np

In [2]:
'''
Test the functionality for bounding code. Bounding box does little to help.
'''
im = openslide.OpenSlide("/Volumes/Datasets/validation/HobI18-618697230393.svs")
dims = im.level_dimensions
RESOLUTION = 2
width = dims[RESOLUTION][0]
height = dims[RESOLUTION][1]

img = im.read_region((0, 0), RESOLUTION, (width, height))
img.show()

In [3]:
type(np.asarray(img))
imgA = np.asarray(img)
sift = cv.xfeatures2d.SIFT_create()
gray = cv.cvtColor(imgA, cv.COLOR_BGR2GRAY)
kp, des = sift.detectAndCompute(gray,None)


In [4]:
print(width)
print(height)

6722
2273


In [14]:
'''
Bin the keypoints into a histogram 
'''
len(kp)
#
x_counter = 0
i = 0
sift_hist = np.zeros(width//100 + 1, dtype=int)

# bin the following:
for k in range(len(kp)):
    x_bin = int(kp[k].pt[0] // 100)
    sift_hist[x_bin] += 1
    
sift_hist

array([ 21,  77, 354, 119,  64,   0,   7,   9,   0,   1,  95, 266, 368,
       359, 400, 458, 421, 491, 388, 646, 405, 209,   0,   0,   2,   0,
         0,   0, 102, 242, 362, 440, 432, 607, 711, 507, 262, 285, 698,
       196,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0])

In [ ]:


# Diagnosis-name pairings
label_reader = csv.DictReader(open("target.csv"))
labels = {}
for row in label_reader:
    labels[row['slide']] = row['target']

# Form bounding tuples
label_reader = csv.DictReader(open("Lymph_Partitions.csv"))
lymph = {}
for row in label_reader:
    name = row['Slide_name'] + ".svs"
    diagnosis = labels[name]
    lymph[name] = (diagnosis, row['L1'], row['L2'], row['L3'], row['L4'])

'''Test manually done bounding boxes.
lymph_bound => ((534, 483), 218, 192)
lymph['HobI18-618697230393.svs'] => ('0', '((199, 397), 217, 215)', '((534, 483), 218, 192)', '((1031, 525), 207, 180)', '')
'''
im = openslide.OpenSlide("/Volumes/Datasets/validation/HobI18-618697230393.svs")
RESOLUTION = 3
dims = im.level_dimensions
slide_width = dims[RESOLUTION][0]
slide_height = dims[RESOLUTION][1]

# print(slide_height)
# print(slide_width)

lymph_bound = eval(lymph['HobI18-618697230393.svs'][2])
print(lymph['HobI18-618697230393.svs']) 

lymph_point = (int(lymph_bound[0][0] / 1280 * slide_width), int(lymph_bound[0][1] / 723 * slide_height))
width = int((lymph_bound[1] / 1280) * slide_width * 4)
height = int((lymph_bound[2] / 723) * slide_height * 4) 


# print(lymph_point)
# print(height)
# print(width)

img = im.read_region(lymph_point, RESOLUTION, (width, height))
img.show()

'''
test the slide_width//lymph_count approach
getcolors() => seems to indicate that partition is happening despite show() only showing grey.
'''
im = openslide.OpenSlide("/Volumes/Datasets/validation/HobI18-618697230393.svs")
RESOLUTION = 2
dims = im.level_dimensions
slide_width = dims[RESOLUTION][0]
slide_height = dims[RESOLUTION][1]
print(slide_width//3)
img = im.read_region((0, 0), RESOLUTION, (slide_width//3, slide_height))
bbox = img.getbbox()
img2 = img.crop(box=bbox)

print(img2.size)
print(img.size)
img.show()
# img2.show()
cols = img.getcolors(img.size[0]*img.size[1])
print(len(cols))
# print(cols)
# print(img.getpalette())
#img.show()
'''
Try using sliding window approach, where if the sliding rectangular bar from the left and right sides of the 
parsed lymph contains x amount of sift features, stop, that is your left and right bound. 

The higher x is, the tighter the bound
'''
type(img)
ar = np.asarray(img)
gray= cv.cvtColor(ar,cv.COLOR_BGR2GRAY)
sift = cv.xfeatures2d.SIFT_create()
kp = sift.detect(gray,None)
img_a=cv.drawKeypoints(gray,kp,np.asarray(img))
cv.imwrite('sift_kp.jpg',img_a)

img_a.show()